we'll take the data from api according to the instruments we've gotten before
convert them into dataframe and save them for furthure use

In [1]:
import requests
import authentication
import pandas as pd

In [2]:
session = requests.Session()

In [3]:
ins_df = pd.read_pickle('./data/instruments.pkl')

In [4]:
our_currencies = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [8]:
a = (ins_df.name.unique())
print(a)
print(len(a))

['AUD_HKD' 'USD_SGD' 'EUR_SEK' 'NZD_CHF' 'AUD_CHF' 'HKD_JPY' 'CAD_JPY'
 'GBP_CHF' 'USD_THB' 'EUR_HKD' 'CHF_JPY' 'GBP_HKD' 'EUR_NZD' 'SGD_CHF'
 'AUD_SGD' 'EUR_JPY' 'USD_CHF' 'USD_TRY' 'GBP_JPY' 'EUR_CZK' 'EUR_TRY'
 'USD_JPY' 'SGD_JPY' 'GBP_ZAR' 'ZAR_JPY' 'TRY_JPY' 'AUD_USD' 'USD_CAD'
 'USD_PLN' 'GBP_AUD' 'AUD_JPY' 'EUR_ZAR' 'EUR_USD' 'EUR_CAD' 'USD_HUF'
 'EUR_CHF' 'NZD_CAD' 'EUR_SGD' 'NZD_JPY' 'CHF_HKD' 'USD_DKK' 'EUR_AUD'
 'USD_ZAR' 'USD_CZK' 'GBP_SGD' 'USD_SEK' 'USD_MXN' 'GBP_USD' 'EUR_DKK'
 'CAD_CHF' 'EUR_GBP' 'CAD_HKD' 'EUR_NOK' 'GBP_CAD' 'NZD_HKD' 'GBP_PLN'
 'EUR_PLN' 'AUD_CAD' 'USD_HKD' 'CAD_SGD' 'GBP_NZD' 'CHF_ZAR' 'USD_CNH'
 'NZD_USD' 'EUR_HUF' 'AUD_NZD' 'USD_NOK' 'NZD_SGD']
68


In [13]:
def fetch_candles(pair_name, count, granularity):
    url = f"{authentication.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, headers=authentication.SECURE_HEADER)
    return response.status_code, response.json()

In [18]:
code, res = fetch_candles("EUR_USD", '20', 'H1')
print(code)
print(res)

200
{'instrument': 'EUR_USD', 'granularity': 'H1', 'candles': [{'complete': True, 'volume': 12493, 'time': '2022-08-26T13:00:00.000000000Z', 'bid': {'o': '1.00318', 'h': '1.00745', 'l': '1.00288', 'c': '1.00618'}, 'mid': {'o': '1.00326', 'h': '1.00752', 'l': '1.00297', 'c': '1.00634'}, 'ask': {'o': '1.00334', 'h': '1.00760', 'l': '1.00305', 'c': '1.00650'}}, {'complete': True, 'volume': 20597, 'time': '2022-08-26T14:00:00.000000000Z', 'bid': {'o': '1.00617', 'h': '1.00893', 'l': '1.00139', 'c': '1.00139'}, 'mid': {'o': '1.00637', 'h': '1.00902', 'l': '1.00146', 'c': '1.00146'}, 'ask': {'o': '1.00657', 'h': '1.00911', 'l': '1.00154', 'c': '1.00154'}}, {'complete': True, 'volume': 9347, 'time': '2022-08-26T15:00:00.000000000Z', 'bid': {'o': '1.00140', 'h': '1.00204', 'l': '0.99835', 'c': '0.99964'}, 'mid': {'o': '1.00146', 'h': '1.00211', 'l': '0.99842', 'c': '0.99972'}, 'ask': {'o': '1.00153', 'h': '1.00218', 'l': '0.99850', 'c': '0.99980'}}, {'complete': True, 'volume': 5710, 'time': '

In [16]:
def get_candles_df(json_response):

    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

In [20]:
df = get_candles_df(res)

In [21]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-08-26T13:00:00.000000000Z,12493,1.00326,1.00752,1.00297,1.00634,1.00318,1.00745,1.00288,1.00618,1.00334,1.00760,1.00305,1.00650
1,2022-08-26T14:00:00.000000000Z,20597,1.00637,1.00902,1.00146,1.00146,1.00617,1.00893,1.00139,1.00139,1.00657,1.00911,1.00154,1.00154
2,2022-08-26T15:00:00.000000000Z,9347,1.00146,1.00211,0.99842,0.99972,1.00140,1.00204,0.99835,0.99964,1.00153,1.00218,0.99850,0.99980
3,2022-08-26T16:00:00.000000000Z,5710,0.99980,1.00030,0.99754,0.99799,0.99972,1.00022,0.99746,0.99791,0.99989,1.00038,0.99762,0.99807
4,2022-08-26T17:00:00.000000000Z,3875,0.99800,0.99832,0.99588,0.99698,0.99792,0.99824,0.99580,0.99691,0.99807,0.99840,0.99595,0.99705


In [22]:
def save_file(candle_df, pair, granularity):
    candle_df.to_pickle(f'historical_data/{pair}_{granularity}.pkl')

In [23]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, 'Error 200')
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [24]:
for p1 in our_currencies:
    for p2 in our_currencies:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, "H1")

EUR_USD loaded 3999 candles from 2022-01-06T16:00:00.000000000Z to 2022-08-29T10:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2022-01-06T14:00:00.000000000Z to 2022-08-29T10:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2022-01-06T08:00:00.000000000Z to 2022-08-29T10:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2022-01-06T17:00:00.000000000Z to 2022-08-29T10:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2022-01-06T16:00:00.000000000Z to 2022-08-29T10:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2022-01-06T17:00:00.000000000Z to 2022-08-29T10:00:00.000000000Z
USD_JPY loaded 3999 candles from 2022-01-06T16:00:00.000000000Z to 2022-08-29T10:00:00.000000000Z
USD_CHF loaded 3999 candles from 2022-01-06T12:00:00.000000000Z to 2022-08-29T10:00:00.000000000Z
USD_CAD loaded 3999 candles from 2022-01-06T17:00:00.000000000Z to 2022-08-29T10:00:00.000000000Z
GBP_USD loaded 3999 candles from 2022-01-06T16:00:00.000000000Z to 2022-08-29T10:00:00.000000000Z
GBP_JPY loaded 3999 